In [1]:
"""
this is the 3rd ipynb
it can create catalog with MGS, dataset
"""

'\nthis is the 3rd ipynb\nit can create catalog with MGS, dataset\n'

In [2]:
import pandas as pd
from pandas import read_parquet
import os
from astropy.io import fits
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from tqdm import  tqdm
from sklearn.model_selection import train_test_split
def match(df_1, df_2, pixel, df1_name):
    """
    match two catalog
    :param df_1:
    :param df_2:
    :return:
    """
    sdss = SkyCoord(ra=df_1.ra * u.degree, dec=df_1.dec * u.degree)
    decals = SkyCoord(ra=df_2.ra * u.degree, dec=df_2.dec * u.degree)
    idx, d2d, d3d = sdss.match_to_catalog_sky(decals)
    max_sep = pixel * 0.262 * u.arcsec
    distance_idx = d2d < max_sep
    sdss_matches = df_1.iloc[distance_idx]
    matches = idx[distance_idx]
    decal_matches = df_2.iloc[matches]
    test = sdss_matches.loc[:].rename(columns={"ra":"%s" % df1_name[0], "dec":"%s" % df1_name[1]})
    test.insert(0, 'ID', range(len(test)))
    decal_matches.insert(0, 'ID', range(len(decal_matches)))
    new_df = pd.merge(test, decal_matches, how="inner", on=["ID"])
    return new_df.drop("ID", axis=1)

In [3]:
# https://doi.org/10.5281/zenodo.4573248
# arXiv:2102.08414v2
# here is Mike Walmsley's catalog
df_auto = read_parquet("/data/renhaoye/MorCG/dataset/gz_decals_auto_posteriors.parquet").rename(columns=lambda x: x.replace("-", "_")) # 头中的-换为_
len(df_auto)

313789

In [4]:
# main galaxy sample raw catalog, 74w sample
with fits.open("/data/renhaoye/MorCG/dataset/VAGC_MGS-m14_1777--20180116.fits") as hdul:
    ra = np.array(hdul[1].data["RA      "])
    dec = np.array(hdul[1].data["DEC     "])
    petro_mag = np.array(hdul[1].data["PETROMAG"][:, 2])
    MGS = pd.DataFrame(np.array((ra, dec, petro_mag)).T, columns=["ra", "dec", "petro_mag"])

In [5]:
# load local file names to pd.Dataframe to match, in_decals, ls-9-south
# these data are in MGS
local_files = os.listdir("/data/renhaoye/MorCG/dataset/in_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1].split(".fits")[0])
        ra.append(t_ra)
        dec.append(t_dec)
in_decals = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|███████████████████████████████████████████████████████████████████████| 361930/361930 [00:00<00:00, 463251.31it/s]


In [6]:
# load local file names to pd.Dataframe to match, out_decals, ls-9-north
# these data are in MGS
local_files = os.listdir("/data/renhaoye/MorCG/dataset/out_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1].split(".fits")[0])
        ra.append(t_ra)
        dec.append(t_dec)
out_decals = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|███████████████████████████████████████████████████████████████████████| 297843/297843 [00:00<00:00, 494446.61it/s]


In [7]:
# overlap = galaxy both in DECaLS and BASS+MzLS, 62497 galaxies
overlap = match(in_decals, out_decals, 2, df1_name=["in_ra", "in_dec"]).drop(columns=["in_ra", "in_dec"])

In [8]:
# from tqdm import  tqdm
# # load local file names to pd.Dataframe to match
# local_files = os.listdir("/data/GZ_Decals/nomerge/")
# ra, dec = [], []
# for i in tqdm(range(len(local_files))):
#     if ".fits" in local_files[i]:
#         t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
#         ra.append(t_ra)
#         dec.append(t_dec)
# local_files = os.listdir("/data/GZ_Decals/merge/")
# for i in tqdm(range(len(local_files))):
#     if ".fits" in local_files[i]:
#         t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
#         ra.append(t_ra)
#         dec.append(t_dec)
# local_dir = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

In [9]:
# use MikeWalmsley's catalog to match Main Galaxy Sample catalog and add petro_mag info
df_auto_mgs = match(MGS, df_auto, 5, ["MGS_ra", "MGS_dec"])

In [10]:
df_auto_mgs

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
0,337.450310,1.266134,16.952042,337.450318,1.266150,J222948.07+011558.1,4.186915,1.915174,4.839483,-20.407906,...,0.018522,0.018522,0.488956,0.451788,0.003392,0.003392,1.0,dr5/png/J222/J222948.07+011558.1.png,146.525273,False
1,343.100541,1.271833,17.530886,343.100545,1.271857,J225224.12+011618.6,3.553056,1.755761,4.028279,-18.125319,...,0.072188,0.072188,0.824894,0.013147,0.007565,0.007565,1.0,dr5/png/J225/J225224.12+011618.6.png,151.577305,False
2,350.324605,1.269114,17.309258,350.324592,1.269100,J232117.90+011608.8,4.028279,1.842611,4.394024,-21.143169,...,0.103261,0.103261,0.810387,0.008353,0.003222,0.003222,1.0,dr5/png/J232/J232117.90+011608.8.png,138.786222,False
3,353.543418,1.271758,17.572227,353.543420,1.271777,J233410.42+011618.4,7.190163,3.434981,7.509485,-20.800541,...,0.476523,0.476523,0.435209,0.015857,0.355152,0.355152,1.0,dr5/png/J233/J233410.42+011618.4.png,148.026903,False
4,353.910231,1.264805,17.464956,353.910235,1.264773,J233538.45+011553.2,7.995928,4.501389,13.797813,-20.697124,...,0.238907,0.238907,0.610592,0.049025,0.042594,0.042594,1.0,dr5/png/J233/J233538.45+011553.2.png,150.268244,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253235,161.961011,9.860375,16.593317,161.960998,9.860361,J104750.63+095137.3,9.472872,3.862174,12.797614,-21.275429,...,0.133771,0.133771,0.523560,0.209438,0.014531,0.014531,1.0,dr5/png/J104/J104750.63+095137.3.png,152.649336,False
253236,162.079082,10.040918,16.656054,162.079057,10.040934,J104818.97+100227.3,3.989568,1.789999,4.746917,-20.202862,...,0.084888,0.084888,0.836097,0.007511,0.003354,0.003354,1.0,dr5/png/J104/J104818.97+100227.3.png,148.478420,False
253237,162.179214,10.066991,16.888079,162.179191,10.067010,J104843.00+100401.1,3.639873,1.755761,4.436659,-18.020481,...,0.097034,0.097034,0.807933,0.005390,0.025462,0.025462,1.0,dr5/png/J104/J104843.00+100401.1.png,152.229218,False
253238,162.439221,9.985815,17.519909,162.439203,9.985830,J104945.40+095908.9,3.819922,1.697413,3.746857,-20.513891,...,0.112341,0.112341,0.814916,0.005653,0.020147,0.020147,1.0,dr5/png/J104/J104945.40+095908.9.png,154.227963,False


In [11]:
# this is local file both in auto catalog and MGS, in DECaLS region
# this is the final dataset
df_loc_decals = match(in_decals, df_auto_mgs, 2, ["del_ra", "del_dec"]).drop(columns=["del_ra", "del_dec"])

In [12]:
df_loc_decals

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,petro_th50,petro_th90,elpetro_absmag_r,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
0,156.404678,32.128263,15.143953,156.404700,32.128272,J102537.12+320741.7,7.807667,3.405226,11.521922,-20.225849,...,0.095087,0.095087,0.609809,0.175285,0.005414,0.005414,1.0,dr5/png/J102/J102537.12+320741.7.png,116.723694,False
1,156.028195,10.281041,15.274831,156.028187,10.281041,J102406.76+101651.7,6.501150,2.819223,9.270042,-22.320599,...,0.096528,0.096528,0.777067,0.007275,0.003151,0.003151,1.0,dr5/png/J102/J102406.76+101651.7.png,111.390991,False
2,41.000599,-8.739704,16.954834,41.000604,-8.739700,J024400.14-084422.9,3.856986,1.878542,4.701300,-18.766365,...,0.038196,0.038196,0.649711,0.275622,0.006129,0.006129,1.0,dr5/png/J024/J024400.14-084422.9.png,140.838770,False
3,236.876370,0.425387,16.856783,236.876419,0.425334,J154730.33+002531.4,8.806542,4.248001,9.377006,-18.320585,...,0.386491,0.386491,0.525614,0.013837,0.240117,0.240117,1.0,dr5/png/J154/J154730.33+002531.4.png,152.437658,False
4,233.329425,7.594146,17.295204,233.329489,7.594180,J153319.06+073538.9,3.434981,1.563657,3.970353,-19.721277,...,0.044695,0.044695,0.797498,0.108880,0.004655,0.004655,1.0,dr5/png/J153/J153319.06+073538.9.png,154.238022,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171430,141.485652,7.017201,17.068510,141.485663,7.017227,J092556.55+070101.9,3.336901,1.497167,4.106831,-20.020388,...,0.092294,0.092294,0.827177,0.024317,0.005223,0.005223,1.0,dr5/png/J092/J092556.55+070101.9.png,150.509885,False
171431,138.774497,3.585291,17.036930,138.774441,3.585325,J091505.87+033507.0,9.129296,4.975659,10.416905,-21.397547,...,0.212240,0.212240,0.600900,0.011914,0.020481,0.020481,1.0,dr5/png/J091/J091505.87+033507.0.png,151.126267,False
171432,129.135154,27.277870,17.037573,129.135177,27.277865,J083632.43+271640.3,5.566817,2.943241,5.648035,-20.498346,...,0.625170,0.625170,0.306107,0.023183,0.521136,0.521136,1.0,dr5/png/J083/J083632.43+271640.3.png,152.656195,False
171433,134.269900,30.152608,17.012079,134.269932,30.152604,J085704.78+300909.3,6.309475,3.193874,7.670071,-20.244175,...,0.680549,0.680549,0.227615,0.034112,0.560014,0.560014,1.0,dr5/png/J085/J085704.78+300909.3.png,152.301367,False


In [13]:
threshold = 0.5
threshold2 = 0.6
merger = df_loc_decals.query('merging_minor_disturbance_fraction > %f  | merging_major_disturbance_fraction > %f | merging_merger_fraction > %f '
                             % (threshold2, threshold2, threshold2))
round = df_loc_decals.query('smooth_or_featured_smooth_fraction >  %f  & how_rounded_round_fraction > %f & merging_none_fraction > %f'
                            % (0.7, 0.8, threshold2))
between = df_loc_decals.query('smooth_or_featured_smooth_fraction >  %f & how_rounded_in_between_fraction > %f & merging_none_fraction > %f'
                              % (0.7, 0.85, threshold2))
cigar = df_loc_decals.query('smooth_or_featured_smooth_fraction > %f & how_rounded_cigar_shaped_fraction > %f & merging_none_fraction > %f'
                            % (threshold, threshold2, threshold2))
edgeOn = df_loc_decals.query('smooth_or_featured_featured_or_disk_fraction > %f & disk_edge_on_yes_fraction > %f & merging_none_fraction > %f'
                             % (threshold, 0.7, threshold2))
noBar = df_loc_decals.query('smooth_or_featured_featured_or_disk_fraction > %f & disk_edge_on_no_fraction > %f & bar_no_fraction > %f  & merging_none_fraction > %f'
                            % (threshold, threshold, 0.7, threshold2))
strongBar = df_loc_decals.query('smooth_or_featured_featured_or_disk_fraction > %f & disk_edge_on_no_fraction > %f &bar_strong_fraction > %f & merging_none_fraction > %f'
                                % (threshold, threshold, 0.55, threshold2))

merger.insert(loc=7, column='label', value=0)
round.insert(loc=7, column='label', value=1)
between.insert(loc=7, column='label', value=2)
cigar.insert(loc=7, column='label', value=3)
edgeOn.insert(loc=7, column='label', value=4)
noBar.insert(loc=7, column='label', value=5)
strongBar.insert(loc=7, column='label', value=6)

merger_train, merger_test = train_test_split(merger, test_size=0.1, random_state=1926)
merger_train, merger_valid = train_test_split(merger_train, test_size=0.1, random_state=1926)
merger_train.insert(loc=7, column='func', value="train")
merger_valid.insert(loc=7, column='func', value="valid")
merger_test.insert(loc=7, column='func', value="test")

round_train, round_test = train_test_split(round, test_size=0.1, random_state=1926)
round_train, round_valid = train_test_split(round_train, test_size=0.1, random_state=1926)
round_train.insert(loc=7, column='func', value="train")
round_valid.insert(loc=7, column='func', value="valid")
round_test.insert(loc=7, column='func', value="test")

between_train, between_test = train_test_split(between, test_size=0.1, random_state=1926)
between_train, between_valid = train_test_split(between_train, test_size=0.1, random_state=1926)
between_train.insert(loc=7, column='func', value="train")
between_valid.insert(loc=7, column='func', value="valid")
between_test.insert(loc=7, column='func', value="test")

cigar_train, cigar_test = train_test_split(cigar, test_size=0.1, random_state=1926)
cigar_train, cigar_valid = train_test_split(cigar_train, test_size=0.1, random_state=1926)
cigar_train.insert(loc=7, column='func', value="train")
cigar_valid.insert(loc=7, column='func', value="valid")
cigar_test.insert(loc=7, column='func', value="test")

edgeOn_train, edgeOn_test = train_test_split(edgeOn, test_size=0.1, random_state=1926)
edgeOn_train, edgeOn_valid = train_test_split(edgeOn_train, test_size=0.1, random_state=1926)
edgeOn_train.insert(loc=7, column='func', value="train")
edgeOn_valid.insert(loc=7, column='func', value="valid")
edgeOn_test.insert(loc=7, column='func', value="test")

noBar_train, noBar_test = train_test_split(noBar, test_size=0.1, random_state=1926)
noBar_train, noBar_valid = train_test_split(noBar_train, test_size=0.1, random_state=1926)
noBar_train.insert(loc=7, column='func', value="train")
noBar_valid.insert(loc=7, column='func', value="valid")
noBar_test.insert(loc=7, column='func', value="test")

strongBar_train, strongBar_test = train_test_split(strongBar, test_size=0.1, random_state=1926)
strongBar_train, strongBar_valid = train_test_split(strongBar_train, test_size=0.1, random_state=1926)
strongBar_train.insert(loc=7, column='func', value="train")
strongBar_valid.insert(loc=7, column='func', value="valid")
strongBar_test.insert(loc=7, column='func', value="test")

merger = pd.concat([merger_train, merger_valid, merger_test])
round = pd.concat([round_train, round_valid, round_test])
between = pd.concat([between_train, between_valid, between_test])
cigar = pd.concat([cigar_train, cigar_valid, cigar_test])
edgeOn = pd.concat([edgeOn_train, edgeOn_valid, edgeOn_test])
noBar = pd.concat([noBar_train, noBar_valid, noBar_test])
strongBar = pd.concat([strongBar_train, strongBar_valid, strongBar_test])

In [26]:
len(merger), len(round), len(between), len(cigar), len(edgeOn), len(noBar), len(strongBar), len(merger)+len(round)+ len(between)+ len(cigar)+ len(edgeOn)+ len(noBar)+ len(strongBar)

(3134, 25481, 25045, 15620, 10381, 8993, 1375, 90029)

In [14]:
len(merger), len(round), len(between), len(cigar), len(edgeOn), len(noBar), len(strongBar), len(merger)+len(round)+ len(between)+ len(cigar)+ len(edgeOn)+ len(noBar)+ len(strongBar)

(2176, 17491, 17028, 9939, 6902, 6088, 1292, 60916)

In [15]:
merger.head(1)

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,func,label,petro_th50,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
40700,29.137778,-0.174993,14.968869,29.13778,-0.174999,J015632.76-001027.4,8.300624,train,0,4.108526,...,0.238437,0.238437,0.614796,0.011264,0.004767,0.004767,1.0,dr5/png/J015/J015632.76-001027.4.png,116.563469,False


In [16]:
df_astre = pd.concat([merger, round, between, cigar, edgeOn, noBar, strongBar])
df_astre.to_csv("/data/renhaoye/MorCG/dataset/astre_catalog.csv")

In [17]:
df_astre

,MGS_ra,MGS_dec,petro_mag,ra,dec,iauname,petro_theta,func,label,petro_th50,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
40700,29.137778,-0.174993,14.968869,29.137780,-0.174999,J015632.76-001027.4,8.300624,train,0,4.108526,...,0.238437,0.238437,0.614796,0.011264,0.004767,0.004767,1.0,dr5/png/J015/J015632.76-001027.4.png,116.563469,False
76580,328.294044,-7.590872,14.993052,328.294052,-7.590833,J215311.30-073533.5,21.499472,train,0,10.275855,...,0.668695,0.668695,0.198326,0.054723,0.567799,0.567799,1.0,dr5/png/J215/J215311.30-073533.5.png,153.106040,False
107830,231.472528,31.287035,16.005224,231.472541,31.287034,J152553.68+311705.5,12.663240,train,0,5.737678,...,0.423254,0.423254,0.461326,0.024532,0.096228,0.096228,1.0,dr5/png/J152/J152553.68+311705.5.png,152.823466,False
113212,237.705288,21.204523,17.322866,237.705304,21.204532,J155049.26+211216.3,6.131171,train,0,3.320829,...,0.196002,0.196002,0.673407,0.010290,0.008712,0.008712,1.0,dr5/png/J155/J155049.26+211216.3.png,154.648073,False
143653,35.760865,-0.142204,16.084639,35.760876,-0.142214,J022302.60-000831.9,10.472028,train,0,4.292131,...,0.908241,0.908241,0.041741,0.018819,0.887330,0.887330,1.0,dr5/png/J022/J022302.60-000831.9.png,133.238959,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122500,217.154929,31.324014,16.456341,217.154947,31.324001,J142837.18+311926.4,10.695449,test,6,4.846077,...,0.838596,0.838596,0.084655,0.037077,0.691115,0.691115,1.0,dr5/png/J142/J142837.18+311926.4.png,154.766501,False
166878,176.018947,-2.817188,15.718610,176.018958,-2.817178,J114404.54-024901.8,7.942868,test,6,3.366474,...,0.623522,0.623522,0.283192,0.042273,0.240184,0.240184,1.0,dr5/png/J114/J114404.54-024901.8.png,140.712653,False
29146,1.861344,-0.705998,15.762888,1.861354,-0.705999,J000726.72-004221.5,10.416905,test,6,3.903157,...,0.917868,0.917868,0.026593,0.020419,0.901038,0.901038,1.0,dr5/png/J000/J000726.72-004221.5.png,115.890519,False
78313,323.143377,-0.369105,17.544783,323.143390,-0.369106,J213234.40-002208.7,4.372860,test,6,1.981007,...,0.747221,0.747221,0.151489,0.055790,0.643406,0.643406,1.0,dr5/png/J213/J213234.40-002208.7.png,145.865006,False


In [23]:
func = ["train", "valid", "test"]
dataset = [df_astre.query("func == 'train'"), df_astre.query("func == 'valid'"), df_astre.query("func == 'test'")]
for i in range(len(dataset)):
    with open("/data/renhaoye/MorCG/dataset/%s.txt" % func[i], "w") as w:
        for j in range(len(dataset[i])):
            label = str(dataset[i].iloc[j].label)
            ra, dec = str(dataset[i].iloc[j, 0]), str(dataset[i].iloc[j, 1]),   # 先拿坐标和标签，这一步仅适用于decals的，sdss匹配的不可以
            path = "/data/renhaoye/MorCG/dataset/in_decals/raw_fits/" + ra + "_" + dec + ".fits"
            if os.path.exists(path):
                w.write(path + " " + str(label)[0] +"\n")
            if func[i] == "train":
                w.write(path.split(".fits")[0] + "_flipped.fits " + str(label)[0] +"\n")
                w.write(path.split(".fits")[0] + "_rotated.fits " + str(label)[0] +"\n")
                w.write(path.split(".fits")[0] + "_shifted.fits " + str(label)[0] +"\n")